# Segmenting and Clustering Assignment
Segmenting and Clustering Neighborhood in Toronto

### Question 1:

In [76]:
!conda install -c conda-forge folium=0.5.0 --yes

import pandas as pd
import time
import requests
import folium
import numpy as np
import geocoder
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [2]:
#Using pandas read_html to get table from wikipedia page
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

#table with text 'Borough' only
match = 'Borough'

df = (pd.read_html(url, match = match, na_values=['Not assigned']))[0]
df.head()

,Postal code,Borough,Neighborhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [3]:
#Drop rows with NaN in Borough column
df.dropna(subset=['Borough'], inplace=True)
df.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [4]:
#Check if Neighborhood column has NaN
df['Neighborhood'].hasnans

False

In [5]:
#Replace '/' by ','
df['Neighborhood'] = df['Neighborhood'].str.replace(' / ', ', ')
df

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [6]:
#Reset Index
df.reset_index(drop=True, inplace=True)
#show shape
df.shape

(103, 3)

### Question 2


In [8]:
#install geocoder
!conda install -c conda-forge geocoder

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          59 KB

The following NEW packages will be INSTALLED:

    geocoder: 1.38.1-py_1 conda-forge
    ratelim:  0.1.6-py_2  conda-forge


geocoder-1.38.1      | 53 KB     | ##################################### | 100% 
ratelim-0.1.6        | 6 KB      | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [9]:
# Add Latitude and Longitude columns

df.insert(3,'Latitude',0)
df.head()

,Postal code,Borough,Neighborhood,Latitude
0,M3A,North York,Parkwoods,0
1,M4A,North York,Victoria Village,0
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",0
3,M6A,North York,"Lawrence Manor, Lawrence Heights",0
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",0


In [10]:
df.insert(4,'Longitude',0)
df.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,0,0
1,M4A,North York,Victoria Village,0,0
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0
3,M6A,North York,"Lawrence Manor, Lawrence Heights",0,0
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",0,0


In [12]:
#loop for each Postal code
for i, postal_code in enumerate(df['Postal code']):
    
    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
        if lat_lng_coords == None:
            time.sleep(1)
            print(".")
    df.iloc[i,3] = lat_lng_coords[0]
    df.iloc[i,4] = lat_lng_coords[1]
    print(i, postal_code, lat_lng_coords)

0 M3A [43.75293455500008, -79.33564142299997]
1 M4A [43.72810248500008, -79.31188987099995]
2 M5A [43.65096410900003, -79.35304116399999]
3 M6A [43.723265465000054, -79.45121077799996]
4 M7A [43.66179000000005, -79.38938999999993]
5 M9A [43.66748067300006, -79.52895286499995]
6 M1B [43.80862623100006, -79.18991284599997]
7 M3B [43.74890000000005, -79.35721999999998]
8 M4B [43.70719267700008, -79.31152927299996]
9 M5B [43.65749059800004, -79.37752923699998]
10 M6B [43.70727872700007, -79.44750009299997]
11 M9B [43.65002250300006, -79.55408903099999]
12 M1C [43.78577865700004, -79.15736763799998]
13 M3C [43.72214339800007, -79.35202341799999]
14 M4C [43.68974004200004, -79.30850701899999]
15 M5C [43.65173364700007, -79.37555358799995]
16 M6C [43.69172991700003, -79.43001279899994]
17 M9C [43.637813150000056, -79.57648363299995]
18 M1E [43.76580607300008, -79.18528434099994]
19 M4E [43.67814827600006, -79.29534930999995]
20 M5E [43.645195888000046, -79.37385548899994]
21 M6E [43.689117566

In [13]:
df.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752935,-79.335641
1,M4A,North York,Victoria Village,43.728102,-79.311890
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.723265,-79.451211
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390


### Question 3:

Foursquare API setup and test call 

In [94]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:


In [95]:
latitude = df.loc[0,'Latitude']
longitude = df.loc[0,'Longitude']
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=&client_secret=&ll=43.75293455500008,-79.33564142299997&v=20180604&radius=500&limit=100'

In [16]:
results = requests.get(url).json()
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
  'name': 'Brookbanks Park',
  'location': {'address': 'Toronto',
   'lat': 43.751976046055574,
   'lng': -79.33214044722958,
   'labeledLatLngs': [{'label': 'display',
     'lat': 43.751976046055574,
     'lng': -79.33214044722958}],
   'distance': 301,
   'cc': 'CA',
   'city': 'Toronto',
   'state': 'ON',
   'country': 'Canada',
   'formattedAddress': ['Toronto', 'Toronto ON', 'Canada']},
  'categories': [{'id': '4bf58dd8d48988d163941735',
    'name': 'Park',
    'pluralName': 'Parks',
    'shortName': 'Park',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-0-4e8d9dcdd5fbbbb6b3003c7b-0'}

Checking which coordinate data will be used

In [19]:
#Download given csv for comparison
!wget -q -O 'geo_data.csv' https://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


NameError: name 'geo_data' is not defined

In [20]:
tor_df = pd.read_csv('geo_data.csv')
tor_df

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


Show map with coordinate data from csv and geocoder.arcgis

In [21]:
# create map of Toronto using latitude and longitude values

latitute = 43.651070
longitude = -79.347015

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)
# add markers to map
for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

for lat, lng, label in zip(tor_df['Latitude'], tor_df['Longitude'], tor_df['Postal Code']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#cc3131',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto) 
    
    
map_toronto

Coordinates looks similar. 
Dataframe df with coordinates from geocoder.arcgis will be used.

Limiting the number of neighborhood in the study. Keeping only those with Toronto text in the Borough: 

In [22]:
tor_data = df[df['Borough'].str.contains('Toronto')].reset_index(drop=True)
tor_data.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657491,-79.377529
3,M5C,Downtown Toronto,St. James Town,43.651734,-79.375554
4,M4E,East Toronto,The Beaches,43.678148,-79.295349


Testing category data extraction process

In [23]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [26]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Variety Store,Food & Drink Shop,43.751974,-79.333114


Function to repeate the same structuring process in all neighborhoods

In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [36]:
toronto_venues = getNearbyVenues(names=tor_data['Neighborhood'],
                                   latitudes=tor_data['Latitude'],
                                   longitudes=tor_data['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town,

In [37]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.650964,-79.353041,Souk Tabule,43.653756,-79.354390,Mediterranean Restaurant
1,"Regent Park, Harbourfront",43.650964,-79.353041,Young Centre for the Performing Arts,43.650825,-79.357593,Performing Arts Venue
2,"Regent Park, Harbourfront",43.650964,-79.353041,SOMA chocolatemaker,43.650622,-79.358127,Chocolate Shop
3,"Regent Park, Harbourfront",43.650964,-79.353041,BATLgrounds,43.647088,-79.351306,Athletics & Sports
4,"Regent Park, Harbourfront",43.650964,-79.353041,Cluny Bistro & Boulangerie,43.650565,-79.357843,French Restaurant


In [38]:
toronto_venues.shape

(1598, 7)

In [39]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,67,67,67,67,67,67
"Brockton, Parkdale Village, Exhibition Place",44,44,44,44,44,44
Business reply mail Processing CentrE,100,100,100,100,100,100
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",65,65,65,65,65,65
Central Bay Street,56,56,56,56,56,56
Christie,12,12,12,12,12,12
Church and Wellesley,79,79,79,79,79,79
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,27,27,27,27,27,27


In [40]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 223 uniques categories.


In [54]:
toronto_venues['Venue Category'].unique()

array(['Mediterranean Restaurant', 'Performing Arts Venue',
       'Chocolate Shop', 'Athletics & Sports', 'French Restaurant', 'Pub',
       'Tech Startup', 'Theater', 'Café', 'Distribution Center',
       'Coffee Shop', 'Bakery', 'Mexican Restaurant', 'Bank',
       'Food Truck', 'Thai Restaurant', 'Seafood Restaurant', 'Trail',
       'Antique Shop', 'Music Venue', 'Restaurant', 'Italian Restaurant',
       'Park', 'Sushi Restaurant', 'Creperie', 'Yoga Studio',
       'Arts & Crafts Store', 'Spa', 'Burrito Place',
       'Fried Chicken Joint', 'Middle Eastern Restaurant', 'Hobby Shop',
       'Discount Store', 'Diner', 'Burger Joint', 'Juice Bar',
       'Bookstore', 'Sandwich Place', 'Pharmacy', 'Bar',
       'College Auditorium', 'Clothing Store', 'Tea Room', 'Hotel',
       'Plaza', 'Comic Shop', 'Art Gallery', 'Ramen Restaurant', 'Lounge',
       'College Rec Center', 'Japanese Restaurant', 'Steakhouse',
       'Shopping Mall', 'Lake', 'Electronics Store', 'Tanning Salon',
     

In [57]:
# one hot encoding
tor_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tor_onehot.insert(0,'NeighborhoodA',toronto_venues['Neighborhood'])

tor_onehot.head()

,NeighborhoodA,Accessories Store,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Group rows by neighborhood and take the mean of the frequency of each category 

In [59]:
tor_grouped = tor_onehot.groupby('NeighborhoodA').mean().reset_index()
tor_grouped

,NeighborhoodA,Accessories Store,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.00,0.014925,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.014925,0.000000,0.000000,0.000000,0.000000,0.014925
1,"Brockton, Parkdale Village, Exhibition Place",0.022727,0.000000,0.00,0.022727,0.000000,0.022727,0.000000,0.000000,0.000000,...,0.022727,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Business reply mail Processing CentrE,0.000000,0.030000,0.00,0.010000,0.000000,0.000000,0.030000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.015385,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.000000,0.000000,0.00,0.000000,0.017857,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.017857,0.017857,0.000000,0.000000,0.000000
5,Christie,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.000000,0.012658,0.00,0.000000,0.000000,0.012658,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.012658
7,"Commerce Court, Victoria Hotel",0.000000,0.040000,0.00,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.010000,0.000000,0.010000
8,Davisville,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.037037,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [60]:
tor_grouped.shape

(39, 224)

Show to 5 venues of each neighborhood

In [61]:
num_top_venues = 5

for hood in tor_grouped['NeighborhoodA']:
    print("----"+hood+"----")
    temp = tor_grouped[tor_grouped['NeighborhoodA'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.04
2  Seafood Restaurant  0.04
3          Restaurant  0.03
4              Bakery  0.03


----Brockton, Parkdale Village, Exhibition Place----
                    venue  freq
0             Coffee Shop  0.09
1                    Café  0.07
2               Gift Shop  0.05
3  Thrift / Vintage Store  0.05
4                   Diner  0.05


----Business reply mail Processing CentrE----
                 venue  freq
0          Coffee Shop  0.07
1                Hotel  0.05
2                 Café  0.04
3  Japanese Restaurant  0.04
4  American Restaurant  0.03


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
               venue  freq
0        Coffee Shop  0.08
1               Café  0.06
2         Restaurant  0.06
3  French Restaurant  0.05
4               Park  0.05


----Central Bay Street----
                       venue  freq


In [62]:
#Function to sort venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [89]:
#Create new dataframe and display it in order of frequency
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = tor_grouped['NeighborhoodA']

for ind in np.arange(tor_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tor_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Seafood Restaurant,Cocktail Bar,Italian Restaurant,Beer Bar,Bakery,Farmers Market,Café,Breakfast Spot,Hotel
1,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Café,Gift Shop,Thrift / Vintage Store,Diner,Dance Studio,French Restaurant,Mexican Restaurant,Boutique,Italian Restaurant
2,Business reply mail Processing CentrE,Coffee Shop,Hotel,Japanese Restaurant,Café,Restaurant,American Restaurant,Asian Restaurant,Theater,Italian Restaurant,Bar
3,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Restaurant,Café,French Restaurant,Park,Gym / Fitness Center,Speakeasy,Italian Restaurant,Bar,Lounge
4,Central Bay Street,Coffee Shop,Middle Eastern Restaurant,Japanese Restaurant,Clothing Store,Plaza,Café,Breakfast Spot,Restaurant,Bubble Tea Shop,Sandwich Place


Running k=means to cluster the neighborhoods

In [90]:
# set number of clusters
kclusters = 5

tor_grouped_clustering = tor_grouped.drop('NeighborhoodA', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tor_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Show Cluster Label with top 10 venues.

In [91]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

tor_merged = tor_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
tor_merged = tor_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

tor_merged.head() # check the last columns!

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041,0,Pub,Café,Athletics & Sports,Theater,Thai Restaurant,Food Truck,Distribution Center,Coffee Shop,Seafood Restaurant,Bank
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390,0,Coffee Shop,Sushi Restaurant,Café,Yoga Studio,Spa,Burger Joint,Burrito Place,Sandwich Place,College Auditorium,Park
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657491,-79.377529,0,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Sandwich Place,Restaurant,Café,Bar,Hotel,Cosmetics Shop,Italian Restaurant
3,M5C,Downtown Toronto,St. James Town,43.651734,-79.375554,0,Café,Coffee Shop,Italian Restaurant,American Restaurant,Gastropub,Cosmetics Shop,Seafood Restaurant,Cocktail Bar,Theater,Creperie
4,M4E,East Toronto,The Beaches,43.678148,-79.295349,0,Health Food Store,Trail,Neighborhood,Coffee Shop,Pub,Church,Deli / Bodega,Creperie,Fish & Chips Shop,Fast Food Restaurant


In [77]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tor_merged['Latitude'], tor_merged['Longitude'], tor_merged['Neighborhood'], tor_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Cluster 1: FOOD - Coffe Shop and Restaurants

In [79]:
tor_merged.loc[tor_merged['Cluster Labels'] == 0, tor_merged.columns[[2] + list(range(5, tor_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",0,Pub,Café,Athletics & Sports,Theater,Thai Restaurant,Food Truck,Distribution Center,Coffee Shop,Seafood Restaurant,Bank
1,"Queen's Park, Ontario Provincial Government",0,Coffee Shop,Sushi Restaurant,Café,Yoga Studio,Spa,Burger Joint,Burrito Place,Sandwich Place,College Auditorium,Park
2,"Garden District, Ryerson",0,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Sandwich Place,Restaurant,Café,Bar,Hotel,Cosmetics Shop,Italian Restaurant
3,St. James Town,0,Café,Coffee Shop,Italian Restaurant,American Restaurant,Gastropub,Cosmetics Shop,Seafood Restaurant,Cocktail Bar,Theater,Creperie
4,The Beaches,0,Health Food Store,Trail,Neighborhood,Coffee Shop,Pub,Church,Deli / Bodega,Creperie,Fish & Chips Shop,Fast Food Restaurant
5,Berczy Park,0,Coffee Shop,Seafood Restaurant,Cocktail Bar,Italian Restaurant,Beer Bar,Bakery,Farmers Market,Café,Breakfast Spot,Hotel
6,Central Bay Street,0,Coffee Shop,Middle Eastern Restaurant,Japanese Restaurant,Clothing Store,Plaza,Café,Breakfast Spot,Restaurant,Bubble Tea Shop,Sandwich Place
7,Christie,0,Grocery Store,Café,Coffee Shop,Candy Store,Park,Playground,Athletics & Sports,Baby Store,Eastern European Restaurant,Electronics Store
8,"Richmond, Adelaide, King",0,Coffee Shop,Café,Restaurant,Clothing Store,Hotel,Sushi Restaurant,Deli / Bodega,Thai Restaurant,Salad Place,American Restaurant
9,"Dufferin, Dovercourt Village",0,Park,Grocery Store,Café,Bank,Pharmacy,Gym / Fitness Center,Gym,Furniture / Home Store,Athletics & Sports,Smoke Shop


#### Cluster 2: KIDS and DOGS - Playground and Park

In [81]:
tor_merged.loc[tor_merged['Cluster Labels'] == 1, tor_merged.columns[[2] + list(range(5, tor_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,North Toronto West,1,Playground,Gym Pool,Park,Garden,Dog Run,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
29,"Moore Park, Summerhill East",1,Playground,Gym,Park,Trail,Yoga Studio,Dog Run,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
33,Rosedale,1,Playground,Grocery Store,Park,Candy Store,Yoga Studio,Dog Run,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm


#### Cluster 3: FITNESS - Gym and Park

In [82]:
tor_merged.loc[tor_merged['Cluster Labels'] == 2, tor_merged.columns[[2] + list(range(5, tor_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Forest Hill North & West,2,Gym / Fitness Center,Park,Restaurant,Yoga Studio,Dog Run,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant


#### Cluster 4: SWIM - Swimming School 

In [83]:
tor_merged.loc[tor_merged['Cluster Labels'] == 3, tor_merged.columns[[2] + list(range(5, tor_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Lawrence Park,3,Swim School,Bus Line,Yoga Studio,Dumpling Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm


#### Cluster 5: HEALTH IT - Health Service and IT

In [84]:
tor_merged.loc[tor_merged['Cluster Labels'] == 4, tor_merged.columns[[2] + list(range(5, tor_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Roselawn,4,Health & Beauty Service,IT Services,Donut Shop,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
